Assignment 5: Caitlin Box

In [1]:
import pygame
import moderngl
import numpy
from math import cos, sin, pi, radians
import glm

pygame.init() # Initlizes its different modules. Display module is one of them.

width = 840
height = 480
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Caitlin Box: Assignment 5")
gl = moderngl.get_context() # Get Previously created context.
gl.info["GL_VERSION"]

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


'4.6.0 NVIDIA 560.94'

In [2]:
diamond_position_data = [
    0.0, 0.8,
    -0.8, 0,
    0.8, 0,
    0.8, 0,
    -0.8, 0,
    0, -0.8
]
diamond_geometry = numpy.array(diamond_position_data).astype("float32")
#Push the Geometry Data to the GPU buffer.
diamond_geometry_buffer = gl.buffer(diamond_geometry) # Create a Vertex buffer object from the data

In [3]:
diamond_vertex_shader_code = '''
#version 330 core

layout (location=0) in vec2 position;

uniform mat2 M;
uniform float scale;
uniform float radius;
uniform float angle; 

void main() {
    float c = cos(radians(angle));
    float s = sin(radians(angle));
    mat2 R = mat2(c, -s, s, c);

    vec2 displacement = R * (radius * vec2(0, 1));
    vec2 P = M * (scale * position + displacement);

    gl_Position = vec4(P, 0.0, 1.0);
}
'''
line_vertex_shader_code = '''
#version 330 core

// add attributes variables here
uniform mat2 M;
uniform float radius;
uniform float angle;

void main() {
    float c = cos(radians(angle));
    float s = sin(radians(angle));
    mat2 R = mat2(c, -s, s, c);
    
    vec2 P = vec2(0);
    if (gl_VertexID > 0) {
        P = radius * R * vec2(0, 1);
    }
    gl_Position = vec4(M * P, 0.0, 1.0);
}
'''

face_vertex_shader_code = '''
#version 330 core

out vec2 f_uv;
uniform mat2 M; 

void main() {
    
    vec2 P, uv;
    switch (gl_VertexID) {
        case 0:
        case 3:
            uv = vec2(0,0);
            P = vec2(-1,-1);
            break;
        case 1:
            uv = vec2(1,0);
            P = vec2(1,-1);
            break; 
        case 2:
        case 4: 
            uv = vec2(1,1);
            P = vec2(1,1);
            break;
        case 5:
            uv = vec2(0,1);
            P = vec2(-1,1);
            break;
    }
    f_uv = uv;
    gl_Position = vec4(P*M,0.0, 1.0);
}
'''

In [4]:
fragment_shader_code = '''
#version 330 core

out vec4 color;
uniform vec3 inColor;

void main() {
   
    color = vec4(inColor,1);
}
'''

face_fragment_shader_code = '''
#version 330 core

out vec4 color;

in vec2 f_uv; 
uniform sampler2D map; 

void main() {

    color = texture(map, f_uv);
}
'''

In [5]:
diamond_program = gl.program(
    vertex_shader= diamond_vertex_shader_code,
    fragment_shader=fragment_shader_code
)

line_program = gl.program(
    vertex_shader= line_vertex_shader_code,
    fragment_shader= fragment_shader_code
)
face_program = gl.program(
    vertex_shader= face_vertex_shader_code,
    fragment_shader= face_fragment_shader_code
)

In [6]:
diamond_renderable = gl.vertex_array(diamond_program,
    [( diamond_geometry_buffer, "2f", "position" ) ]
)

line_renderable = gl.vertex_array(line_program,
    []
)
face_renderable = gl.vertex_array(face_program, [])

In [7]:
texture_img = pygame.image.load("clockFace.png")
texture_data = pygame.image.tobytes(texture_img, "RGB", True)
faceTexture = gl.texture(texture_img.get_size(), data = texture_data, components=3)
faceSampler = gl.sampler(texture=faceTexture, filter=(gl.LINEAR, gl.LINEAR))

In [8]:
texture_img.get_size()

(1660, 1655)

In [9]:
running = True
clock = pygame.time.Clock()
alpha = 0

while running:   
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    # event.key == 27 means Escape key is pressed.
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == 27):
            running = False
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
    aspect = width/height
    # create the aspect ratio correction matrix
    
    M_aspect = glm.mat2(1/aspect, 0, 0, 1) if aspect > 1 else glm.mat2(1, 0, 0, aspect)

    #Create the rotation matrix
    # ... Part of assignment 4
    # M = M_aspect * M_rotate
    M = M_aspect
    
    ### Add render code below
    # Clear the display window with the specified R, G, B values using function ctx.clear(R, G, B)
    gl.clear(0.5,0.5,0.0)

    faceSampler.use(0)
    faceTexture.use(location=0)
    face_program["map"] = 0
    face_program["M"].write(M)
    face_renderable.render(moderngl.TRIANGLES, vertices=6)
    
    # Make one or more Render Calls to instruct the GPU to render by executing the shader program with the provided data.

    diamond_program["M"].write(M)
    
    diamond_program["scale"] = 0.1
    diamond_program["inColor"] = 1, 1, 0
    diamond_program ['angle'] = alpha
    diamond_program["radius"] = 0.8
    diamond_renderable.render()
    
    diamond_program["radius"] = 0.0
    diamond_program["angle"] = 0.0
    diamond_renderable.render()
    
    line_program["M"].write(M)
    line_program["radius"] = 0.8
    line_program["inColor"] = 0, 0, 0
    line_program ['angle'] = alpha
    line_renderable.render(moderngl.LINES, vertices=2)  
    
    pygame.display.flip()
    clock.tick(60)  # limits FPS to 10
    alpha = alpha + 0.1
    if alpha > 360:
        alpha = 0

pygame.display.quit()